 1. [Fuzzy Sugeno](#1.-Fuzzy-Sugeno)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

OSError: [WinError 127] The specified procedure could not be found: 'C:\\Users\\lenovo\\miniconda3\\lib\\site-packages\\numpy\\.libs'

#### nim = 190602043
gaji = 43 * 20000
masa_kerja = 7 + 3

In [ ]:
data = pd.DataFrame({
    'Gaji' : [i*20000 for i in range(100)],
    'Masa_Kerja' : [(i % 10) + 7 for i in range(100)]
})

In [ ]:
data[:30]

In [ ]:
print('Gaji:', data.Gaji[43])
print('Masa_Kerja:', data.Masa_Kerja[43])

# 1. Fuzzy Sugeno

In [ ]:
g_rendah_x = [0, 300, 800, 2300]
g_rendah_y = [1, 1, 0, 0]
g_sedang_x = [0, 500, 1000, 1500, 2300]
g_sedang_y = [0, 0, 1, 0, 0]
g_tinggi_x = [0, 1000, 2000, 2300]
g_tinggi_y = [0, 0, 1, 1]

mk_baru_x = [0, 5, 15, 30]
mk_baru_y = [1, 1, 0, 0]
mk_lama_x = [0, 10, 25, 30]
mk_lama_y = [0, 0, 1, 1]

fig, ax = plt.subplots(figsize=(20, 5), ncols=2)
ax[0].plot(g_rendah_x, g_rendah_y)
ax[0].plot(g_sedang_x, g_sedang_y)
ax[0].plot(g_tinggi_x, g_tinggi_y)
ax[0].set_xticks([300, 500, 800, 1000, 1500, 2000])
ax[0].set_yticks([0, 1])
ax[0].set_title('Gaji')
ax[0].set_xlabel('Gaji (x1000 rupiah/bulan)')
ax[0].set_ylabel('μ(x)')

ax[0].grid(True)

ax[1].plot(mk_baru_x, mk_baru_y)
ax[1].plot(mk_lama_x, mk_lama_y)
ax[1].set_xticks([0, 5, 10, 15, 25])
ax[1].set_yticks([0, 1])
ax[1].set_title('Masa Kerja')
ax[1].set_xlabel('Masa Kerja (Tahun)')
ax[1].set_ylabel('μ(x)')
ax[1].grid(True)

plt.show()

In [ ]:
def gaji_rendah_value(gaji):
    x = gaji
    a, b = 300000, 800000
    if x < a: 
        nilai = 1
    if a <= x and x <= b: 
        nilai = (b - x) / (b - a)
    if x > b:
        nilai = 0
    return nilai
def gaji_sedang_value(gaji):
    x = gaji
    a, b, c = 500000, 1000000, 1500000
    if x < a or x > b:
        nilai = 0
    if a <= x and x <= b:
        nilai = (x - a) / (b - a)
    if b <= x and x <= c:
        nilai = (c - x) / (c - b)
    return nilai
def gaji_tinggi_value(gaji):
    x = gaji
    a, b = 1000000, 2000000
    if x < a:
        nilai = 0
    if a <= x and x <= b:
        nilai = (x - a) / (b - a)
    if x > b:
        nilai = 1
    return nilai

In [ ]:
def masa_kerja_baru_value(masa_kerja):
    x = masa_kerja
    a, b = 5, 15
    if x < a:
        nilai = 1
    if a <= x and x <= b:
        nilai = (b - x) / (b - a)
    if x > b:
        nilai = 0
    return nilai
def masa_kerja_lama_value(masa_kerja):
    x = masa_kerja
    a, b = 10, 25
    if x < a:
        nilai = 0
    if a <= x and x <= b:
        nilai = (x - a) / (b - a)
    if x > b:
        nilai = 1
    return nilai

## Fuzzification

In [ ]:
def gaji_fuzzification(gaji_value):
    gaji_set = np.array([])
    gaji_set = np.append(gaji_set, gaji_rendah_value(gaji_value))
    gaji_set = np.append(gaji_set, gaji_sedang_value(gaji_value))
    gaji_set = np.append(gaji_set, gaji_tinggi_value(gaji_value))
    return gaji_set
def masa_kerja_fuzzification(masa_kerja_value):
    masa_kerja_set = np.array([])
    masa_kerja_set = np.append(masa_kerja_set, masa_kerja_baru_value(masa_kerja_value))
    masa_kerja_set = np.append(masa_kerja_set, masa_kerja_lama_value(masa_kerja_value))
    return masa_kerja_set

## Inference

In [ ]:
def inference_sugeno(gaji_set, masa_kerja_set, gaji, masa_kerja):
    inference, z = np.array([]), np.array([])
    
    inference = np.append(inference, np.min(np.append(gaji_set[0], masa_kerja_set[0])))
    inference = np.append(inference, np.min(np.append(gaji_set[0], masa_kerja_set[1])))
    inference = np.append(inference, np.min(np.append(gaji_set[1], masa_kerja_set[0])))
    inference = np.append(inference, np.min(np.append(gaji_set[1], masa_kerja_set[1])))
    inference = np.append(inference, np.min(np.append(gaji_set[2], masa_kerja_set[0])))
    inference = np.append(inference, np.min(np.append(gaji_set[2], masa_kerja_set[1])))
    
    z = np.append(z, gaji)
    z = np.append(z, gaji + 150000)
    z = np.append(z, 1350000)
    z = np.append(z, 1600000 + 40000 * 10)
    z = np.append(z, 1500000)
    z = np.append(z, 1500000 + 100000 * 10)
    
    return pd.DataFrame({
        "Inference": inference,
        "Z": z
    })

## Defuzzification

In [ ]:
def defuzzification(inference):
    multiplier = np.sum(np.array([inference.Inference[i] * inference.Z[i] for i in range(len(inference))]))
    divider = np.sum(np.array([inference.Inference[i]for i in range(len(inference))]))
    return multiplier / divider

## Main

In [ ]:
def result(data):
    gaji = data.Gaji
    masa_kerja = data.Masa_Kerja
    result = np.array([])
    for i in range(len(data)):
        gaji_fuzzy = gaji_fuzzification(gaji[i])
        masa_kerja_fuzzy = masa_kerja_fuzzification(masa_kerja[i])
        inference = inference_sugeno(gaji_fuzzy, masa_kerja_fuzzy, gaji[i], masa_kerja[i])
        result = np.append(result, defuzzification(inference))
    data['Bonus'] = result
    return data

In [ ]:
result = result(data)

In [ ]:
pd.options.display.float_format = '{:.2f}'.format

In [3]:
result.iloc[4]

NameError: name 'result' is not defined

# Fuzzy Tsukamoto

In [ ]:
kehadiran_rendah_x = [0, 1, 20, 31]
kehadiran_rendah_y = [1, 1, 0, 0]
kehadiran_tinggi_x = [0, 15, 30, 31]
kehadiran_tinggi_y = [0, 0, 1, 1]

bonus_sedikit_x = [0, 860, 1700, 4100]
bonus_sedikit_y = [1, 1, 0, 0]
bonus_banyak_x = [0, 1300, 3900, 4100]
bonus_banyak_y = [0, 0, 1, 1]

fig, ax = plt.subplots(figsize=(20, 10), nrows=2, ncols=2)
ax[0][0].plot(g_rendah_x, g_rendah_y)
ax[0][0].plot(g_sedang_x, g_sedang_y)
ax[0][0].plot(g_tinggi_x, g_tinggi_y)
ax[0][0].set_xticks([300, 500, 800, 1000, 1500, 2000])
ax[0][0].set_yticks([0, 1])
ax[0][0].set_title('Gaji')
ax[0][0].set_xlabel('Gaji (x1000 rupiah/bulan)')
ax[0][0].set_ylabel('μ(x)')
ax[0][0].grid(True)

ax[0][1].plot(mk_baru_x, mk_baru_y)
ax[0][1].plot(mk_lama_x, mk_lama_y)
ax[0][1].set_xticks([0, 5, 10, 15, 25])
ax[0][1].set_yticks([0, 1])
ax[0][1].set_title('Masa Kerja')
ax[0][1].set_xlabel('Masa Kerja (Tahun)')
ax[0][1].set_ylabel('μ(x)')
ax[0][1].grid(True)

ax[1][0].plot(kehadiran_rendah_x, kehadiran_rendah_y)
ax[1][0].plot(kehadiran_tinggi_x, kehadiran_tinggi_y)
ax[1][0].set_xticks([1, 15, 20, 30])
ax[1][0].set_yticks([0, 1])
ax[1][0].set_title('Kehadiran')
ax[1][0].set_xlabel('Kehadiran')
ax[1][0].set_ylabel('μ(x)')
ax[1][0].grid(True)

ax[1][1].plot(bonus_sedikit_x, bonus_sedikit_y)
ax[1][1].plot(bonus_banyak_x, bonus_banyak_y)
ax[1][1].set_xticks([860, 1300, 1700, 3900])
ax[1][1].set_yticks([0, 1])
ax[1][1].set_title('bonus')
ax[1][1].set_xlabel('bonus')
ax[1][1].set_ylabel('μ(x)')
ax[1][1].grid(True)

plt.show()

In [ ]:
def kehadiran_rendah_value(bonus):
    x = bonus
    a, b = 860, 1700
    if x < a: 
        nilai = 1
    if a <= x and x <= b: 
        nilai = (b - x) / (b - a)
    if x > b:
        nilai = 0
    return nilai
def kehadiran_tinggi_value(bonus):
    x = bonus
    a, b = 1300, 3900
    if x < a:
        nilai = 0
    if a <= x and x <= b:
        nilai = (x - a) / (b - a)
    if x > b:
        nilai = 1
    return nilai

In [ ]:
def bonus_banyak_value(bonus):
    x = bonus
    a, b = 860, 1700
    if x < a: 
        nilai = 1
    if a <= x and x <= b: 
        nilai = (b - x) / (b - a)
    if x > b:
        nilai = 0
    return nilai
def bonus_banyak_value(bonus):
    x = bonus
    a, b = 1300, 30
    if x < a:
        nilai = 0
    if a <= x and x <= b:
        nilai = (x - a) / (b - a)
    if x > b:
        nilai = 1
    return nilai

## Fuzzification

In [ ]:
def kehadiran_fuzzification(kehadiran_value):
    kehadiran_set = np.array([])
    kehadiran_set = np.append(kehadiran_set, kehadiran_rendah_value(kehadiran_value))
    kehadiran_set = np.append(kehadiran_set, kehadiran_tinggi_value(kehadiran_value))
    return kehadiran_set
def bonus_fuzzification(bonus_value):
    bonus_set = np.array([])
    bonus_set = np.append(bonus_set, bonus_sedikit_value(bonus_value))
    bonus_set = np.append(bonus_set, bonus_banyak_value(bonus_value))
    return bonus_set

## Inference

In [ ]:
def inference_f(gaji_set, masa_kerja_set, gaji, masa_kerja):
    inference, z = np.array([]), np.array([])
    
    inference = np.append(inference, np.min(np.append(gaji_set[0], masa_kerja_set[0])))
    inference = np.append(inference, np.min(np.append(gaji_set[0], masa_kerja_set[1])))
    inference = np.append(inference, np.min(np.append(gaji_set[1], masa_kerja_set[0])))
    inference = np.append(inference, np.min(np.append(gaji_set[1], masa_kerja_set[1])))
    inference = np.append(inference, np.min(np.append(gaji_set[2], masa_kerja_set[0])))
    inference = np.append(inference, np.min(np.append(gaji_set[2], masa_kerja_set[1])))
    
    z = np.append(z, gaji)
    z = np.append(z, gaji + 150000)
    z = np.append(z, 1350000)
    z = np.append(z, 1600000 + 40000 * 10)
    z = np.append(z, 1500000)
    z = np.append(z, 1500000 + 100000 * 10)
    
    return pd.DataFrame({
        "Inference": inference,
        "Z": z
    })

## Defuzzification

In [ ]:
def defuzzification(inference):
    multiplier = np.sum(np.array([inference.Inference[i] * inference.Z[i] for i in range(len(inference))]))
    divider = np.sum(np.array([inference.Inference[i]for i in range(len(inference))]))
    return multiplier / divider

## Main

In [ ]:
def result(data):
    gaji = data.Gaji
    masa_kerja = data.Masa_Kerja
    result = np.array([])
    for i in range(len(data)):
        gaji_fuzzy = gaji_fuzzification(gaji[i])
        masa_kerja_fuzzy = masa_kerja_fuzzification(masa_kerja[i])
        inference = inference_sugeno(gaji_fuzzy, masa_kerja_fuzzy, gaji[i], masa_kerja[i])
        result = np.append(result, defuzzification(inference))
    data['Bonus'] = result
    return data

In [ ]:
result = result(data)

In [ ]:
pd.options.display.float_format = '{:.2f}'.format

In [ ]:
result.iloc[4]

In [ ]:
NYERAHHH SAVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVVV NGANTUKK AKUUUUUUUUUUUUU,, yws mas gpp makasih yo